<a href="https://colab.research.google.com/github/GeorgiyAleksanyan/14-332-472-01-ROBOTICS-COMP-VISION-Classify-Dog-vs-Cat-Kaggle-dataset-with-two-different-networks/blob/main/ga360_RCV_14_332_472_Project_1_Question_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classify Dog vs Cat Kaggle dataset with two different networks** Fine-tune a pre-trained network for the dog vs. cat classification problem. Report the confusion matrix, the accuracy, the f-score, precision and recall of your classifier. Write a paragraph describing your results and methods.

In [1]:
!pip install torch~=2.1.0 torchvision pytorch-lightning
!pip install validators matplotlib
!pip install -U 'torch_xla>=2.1'
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: uritemplate
    Found existing installation: uritemplate 4.1.1
    Uninstalling uritemplate-4.1.1:
      Successfully uninstalled uritemplate-4.1.1
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.11.1
    Uninstalling google-api-core-2.11.1:
      Successfully uninstalled google-api-core-2.11.1
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
E

In [14]:
!pip install cloud-tpu-client
!pip install torch-xla

In [15]:
import torch
from torch import nn
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tqdm.auto import tqdm

import torch, torchvision
from torchvision.transforms import ToTensor, Resize
from torchvision import transforms
import os
from PIL import Image
from torch.utils.data import DataLoader
import torch.nn as nn
from torch import optim
from tqdm import tqdm
import copy
import numpy as np
from torch.utils.data import Dataset
import requests
from io import BytesIO
import pytorch_lightning as pl
import torch_xla.core.xla_model as xm

In [18]:
torch.__version__
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

train_folder = '/content/drive/MyDrive/Colab Notebooks/Project 1/dogs vs cats Kaggle Data/train3'
test_folder = '/content/drive/MyDrive/Colab Notebooks/Project 1/dogs vs cats Kaggle Data/test1'

class CatDogDataset(Dataset):
    def __init__(self, train_dir, transform=None):
        self.train_dir = train_dir
        self.transform = transform
        self.images = os.listdir(train_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_path = os.path.join(self.train_dir, self.images[index])
        label = self.images[index].split(".")[0]
        label = 0 if label == 'cat' else 1

        image = np.array(Image.open(image_path))  # Convert to NumPy array
        image = Image.fromarray(image)  # Convert to PIL image

        if self.transform is not None:
            image = self.transform(image)

        return image, label

# Create datasets and data loaders
train_dataset = CatDogDataset(train_folder, transform=transform)
test_dataset = CatDogDataset(test_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, num_workers = 4)
test_loader = DataLoader(test_dataset, batch_size=16, num_workers = 4)

# Define a PyTorch Lightning module for binary classification
class ResNetBinaryClassification(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
        # Replace the final fully connected layer for binary classification
        self.model.fc = nn.Linear(self.model.fc.in_features, 2)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        self.log('train_loss', loss)  # Logging for TensorBoard
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

# Create an instance of the ResNetBinaryClassification model
model = ResNetBinaryClassification()

# Create a trainer for training the model
trainer = pl.Trainer(
    accelerator= 'cpu',
    #tpu_cores = 8,
    #gpus=1,  # Use GPU if available
    max_epochs=1  # Increase the number of epochs as needed
)

# Train the model
trainer.fit(model, train_loader)
# Save the trained model
trainer.save_checkpoint("resnet18KaggleCatDog.pt")
# Evaluate the model
model.eval()
true_labels, predicted_labels = [], []

with torch.no_grad():
    for batch in tqdm(test_loader, total=len(test_loader)):
        x, y = batch
        true_labels.extend(y.cpu())
        logits = model(x).cpu()
        predicted_labels.extend(logits.argmax(dim=1))

# Print classification report
print(classification_report(true_labels, predicted_labels, target_names=['cat', 'dog'], digits=3))

# Create a confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Calculate accuracy, F1-score, precision, and recall
accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')

print("Confusion Matrix:")
print(conf_matrix)
print(f"Accuracy: {accuracy:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Using cache found in /root/.cache/torch/hub/pytorch_vision_main
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type             | Params
---------------------------------------------
0 | model   | ResNet           | 11.2 M
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:104: Total length of `DataLoader` across ranks is zero. Please make sure this was your intention.
/usr/local/lib/python3.10/dist-packages/pytorch_light

              precision    recall  f1-score   support

         cat      0.000     0.000     0.000         0
         dog      1.000     0.519     0.683     12500

    accuracy                          0.519     12500
   macro avg      0.500     0.260     0.342     12500
weighted avg      1.000     0.519     0.683     12500

Confusion Matrix:
[[   0    0]
 [6012 6488]]
Accuracy: 0.52
F1-Score: 0.68
Precision: 1.00
Recall: 0.52


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
